<a href="https://colab.research.google.com/github/mcHan18/Mengchen-Wang/blob/master/tensorflw_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#2. Get the file   #在此处吧你的文件id改进去
downloaded = drive.CreateFile({'id':'1E0H96iAkTurDE80UPeNvhwscjdjxO16I'}) # replace the id with id of file you want to access
#输入你的文件名字
downloaded.GetContentFile('movies.csv') 

downloaded = drive.CreateFile({'id':'18TnXQMVpjpdFEC115Ogrcd_B-nsNjQOQ'}) # replace the id with id of file you want to access
#输入你的文件名字
downloaded.GetContentFile('ratings.csv') 


In [ ]:
#3. Read file as panda dataframe

ratings_df = pd.read_csv('ratings.csv')
ratings_df.tail()
movies_df = pd.read_csv('movies.csv')
movies_df['movieRow'] = movies_df.index #添加电影行数

In [ ]:
ratings_df.tail()

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [ ]:
movies_df.tail()

,movieId,title,genres,movieRow
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,9737
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,9738
9739,193585,Flint (2017),Drama,9739
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,9740
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,9741


In [ ]:
movies_df = movies_df[['movieRow','movieId','title']]
movies_df.to_csv('moviesProcessed.csv',index = False,header = True,encoding = 'utf-8')

In [ ]:
movies_df.tail()

,movieRow,movieId,title
9737,9737,193581,Black Butler: Book of the Atlantic (2017)
9738,9738,193583,No Game No Life: Zero (2017)
9739,9739,193585,Flint (2017)
9740,9740,193587,Bungo Stray Dogs: Dead Apple (2018)
9741,9741,193609,Andrew Dice Clay: Dice Rules (1991)


In [ ]:
ratings_df = pd.merge(ratings_df,movies_df,on = 'movieId')

In [ ]:
ratings_df = ratings_df[['userId','movieRow','rating']]
ratings_df.to_csv('ratingsProcessed.csv',index = False,header = True,encoding = 'utf-8')

In [ ]:
ratings_df.tail()

,userId,movieRow,rating
100831,610,9325,2.5
100832,610,9330,4.5
100833,610,9342,3.0
100834,610,9389,3.5
100835,610,9390,3.5


In [ ]:
userNo = ratings_df['userId'].max()+1
movieNo = ratings_df['movieRow'].max()+1
rating = np.zeros((movieNo,userNo))

In [ ]:
rating

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
flag = 0 #记录处理进度
ratings_df_length = np.shape(ratings_df)[0]
for index,row in ratings_df.iterrows():#获取ratings_df的每一行
        rating[int(row['movieRow']),int(row['userId'])] = row['rating']
        flag += 1 #表示处理完一行
        print('processed %d,%d left'%(flag,ratings_df_length-flag))

流式输出内容被截断，只能显示最后 5000 行内容。
processed 95837,4999 left
processed 95838,4998 left
processed 95839,4997 left
processed 95840,4996 left
processed 95841,4995 left
processed 95842,4994 left
processed 95843,4993 left
processed 95844,4992 left
processed 95845,4991 left
processed 95846,4990 left
processed 95847,4989 left
processed 95848,4988 left
processed 95849,4987 left
processed 95850,4986 left
processed 95851,4985 left
processed 95852,4984 left
processed 95853,4983 left
processed 95854,4982 left
processed 95855,4981 left
processed 95856,4980 left
processed 95857,4979 left
processed 95858,4978 left
processed 95859,4977 left
processed 95860,4976 left
processed 95861,4975 left
processed 95862,4974 left
processed 95863,4973 left
processed 95864,4972 left
processed 95865,4971 left
processed 95866,4970 left
processed 95867,4969 left
processed 95868,4968 left
processed 95869,4967 left
processed 95870,4966 left
processed 95871,4965 left
processed 95872,4964 left
processed 95873,4963 left
processed 9

In [ ]:
#将构建record矩阵并将矩阵中的Boolean值改成相应的数值
record = rating>0 
record = np.array(record,dtype = int)

In [ ]:
record

array([[0, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
def normalizeRatings(rating,record):#缩放评分矩阵范围
    m,n = rating.shape#m为电影数量，n为用户数量
    rating_mean = np.zeros((m,1))#初始化电影评分平均值为0
    rating_norm = np.zeros((m,n))#保存处理后的数据
    for i in range(m):
        idx = record[i,:] != 0 #获取每部电影评分用户的下标，每部电影的评分
        rating_mean[i] = np.mean(rating[i,idx])#表示第i行已经评过分的用户的平均值
        rating_norm[i,idx] -= rating_mean[i]
    return rating_norm,rating_mean

In [ ]:
import tensorflow.compat.v1 as tf

rating_norm,rating_mean = normalizeRatings(rating,record) #由于数据中的某些行有这里会出现警告
#处理nan
rating_norm = np.nan_to_num(rating_norm)
rating_mean = np.nan_to_num(rating_mean)
#假设一共有10类电影
num_features = 10
#初始化电影内容矩阵和用户喜好矩阵，产生的参数都是随机数并且是正态分布的
X_parameters = tf.Variable(tf.random_normal([movieNo,num_features],stddev = 0.35))
Theta_parameters = tf.Variable(tf.random_normal([userNo,num_features],stddev = 0.35))
# 构建损失函数
loss = 1/2 * tf.reduce_sum(((tf.matmul(X_parameters,Theta_parameters,transpose_b = True)- rating_norm)*record)**2)+ 1/2 *(tf.reduce_sum(X_parameters**2) + tf.reduce_sum(Theta_parameters**2))
#将X_parameters，Theta_parameters矩阵相乘相乘之前将Theta_parameters转置
#创建优化器和优化目标
optimizer = tf.train.AdamOptimizer(1e-4)
train = optimizer.minimize(loss)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
tf.summary.scalar('loss',loss)
summaryMerged = tf.summary.merge_all()
filename = 'movie_tensorboard'
writer = tf.summary.FileWriter(filename)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(5000):
    _,movie_summary = sess.run([train,summaryMerged])
    writer.add_summary(movie_summary, i)

In [ ]:
Current_X_parameters,Current_Theta_parameters = sess.run([X_parameters,Theta_parameters])
predicts = np.dot(Current_X_parameters,Current_Theta_parameters.T)+rating_mean
errors = np.sqrt(np.sum((predicts-rating)**2))

In [ ]:
user_id = input('请输入要推荐的用户编号：')
sortedResult = predicts[:,int(user_id)].argsort()[::-1]#获取该用户的电影评分列表
idx =0
print('为该用户提供的推荐列表：'.center(80,'='))
for i in sortedResult:
    print('评分：%.2f,电影名：%s'%(predicts[i,int(user_id)],movies_df.iloc[i]['title']))
    idx +=1
    if idx == 20:break

请输入要推荐的用户编号：20
==================================为该用户提供的推荐列表：==================================
评分：5.81,电影名：Madame Sousatzka (1988)
评分：5.51,电影名：Year of the Horse (1997)
评分：5.41,电影名：Hellbenders (2012)
评分：5.24,电影名：Tom and Jerry: Shiver Me Whiskers (2006)
评分：5.23,电影名：Ice Age: The Great Egg-Scapade (2016)
评分：5.12,电影名：Scooby-Doo! Curse of the Lake Monster (2010)
评分：5.08,电影名：Asterix and the Vikings (Astérix et les Vikings) (2006)
评分：5.08,电影名：Scooby-Doo! and the Samurai Sword (2009)
评分：5.00,电影名：Monster Squad, The (1987)
评分：4.97,电影名：Kung Fu Panda: Secrets of the Masters (2011)
评分：4.92,电影名：7 Faces of Dr. Lao (1964)
评分：4.90,电影名：Tom and Jerry: A Nutcracker Tale (2007)
评分：4.89,电影名：Garfield's Pet Force (2009)
评分：4.80,电影名：What Love Is (2007)
评分：4.79,电影名：UnHung Hero (2013)
评分：4.76,电影名：George Carlin: Back in Town (1996)
评分：4.73,电影名：Mephisto (1981)
评分：4.73,电影名：Dragons: Gift of the Night Fury (2011)
评分：4.68,电影名：Phir Hera Pheri (2006)
评分：4.67,电影名：Red Flag (2012)
